In [1]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
from tensorflow import lite as tflite
import time

import h5py
import os
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Convolution1D, MaxPooling2D, Convolution2D, DepthwiseConv2D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import backend as K
from tensorflow.keras.activations import relu, softmax

from keras.callbacks import ModelCheckpoint
from keras.callbacks import RemoteMonitor
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import f1_score


ModuleNotFoundError: No module named 'tensorflow'

In [43]:
wav_root = '../data/audio_1sec/'

In [44]:
labels = pd.read_csv('../labels/audio_1sec.csv',  header=None, names=["path", "yes", "no", "not_sure","subject_set"])
labels['path'] = labels['path'].astype(str) + '.wav'

# calculate the percentage of "yes" responses for each label
labels['calc'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure'])

# filter the labels to get only the ones with a definite answer
filtered_labels = labels[(labels['yes'] + labels['no']) > 0]

# sort the filtered labels by the calculated percentage in descending order
sorted_labels = filtered_labels.sort_values(by='calc', ascending=False)

# select the highest half of the labels where calc > 0.5
num_high = len(sorted_labels[sorted_labels['calc'] > 0.5])
selected_high = sorted_labels[:num_high]

# select the lowest half of the labels where calc < 0.5
num_low = len(sorted_labels[sorted_labels['calc'] < 0.5])
selected_low = sorted_labels[-num_low:]

# concatenate the selected high and low labels and shuffle them randomly
labels = pd.concat([selected_high, selected_low], axis=0).sample(frac=1).reset_index(drop=True)

# calculate the binary labels (1 for "yes", 0 for "no")
labels['res'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure']) >= 0.5

# create y from the selected labels
n = len(labels)
y = np.zeros((n, 2))
y[:,1] = np.array(labels['res']).astype(int)
y[:,0] = 1 - y[:,1]

feature_type = 'mfcc'


In [45]:
# labels = pd.read_csv('../labels/audio_1sec.csv',  header=None, names=["path", "yes", "no", "not_sure","subject_set"])
# labels['path'] = labels['path'].astype(str) + '.wav'
# labels['calc'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure']) 



# labels['res'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure']) >= 0.5


# n = len(labels.index.values.tolist())

# y = np.zeros((n, 2))

# y[:,1] = np.array(np.array(labels['res']).astype(int)).astype(int)
# y[:,0] = 1-y[:,1].astype(int)

# feature_type = 'mfcc'


In [46]:
# Set window length in seconds
win_len = 0.1

# Set FFT size based on window length and sampling rate
nfft = int(win_len * 8000)

# Set number of MFCCs per frame
n_mfcc = 40

# Load list of audio file paths
wav_path = wav_root + labels["path"]
file_list = list(wav_path)

# Initialize feature matrix
spec_matrix = np.zeros((len(file_list)*49, n_mfcc))

# Extract MFCC features from each file and store in feature matrix
for i, file_path in enumerate(file_list):
    wav, fs = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=wav, sr=fs, n_mfcc=n_mfcc, n_fft=nfft*4, hop_length=nfft)
    
    # Pad or truncate the feature matrix to have shape (49, n_mfcc)
    if mfccs.shape[1] < 49:
        mfccs = np.pad(mfccs, ((0, 0), (0, 49 - mfccs.shape[1])), mode='constant')
    elif mfccs.shape[1] > 49:
        mfccs = mfccs[:, :49]

    # Store MFCC features in the spec_matrix
    spec_matrix[i*49:(i+1)*49,:] = mfccs.T

ModuleNotFoundError: No module named 'librosa.core.audio'

In [4]:
# win_len = 0.1

# nfft = int(win_len * 8000)
# wav_path = wav_root + labels["path"]



# wav, fs = librosa.load(wav_path.iloc[0],sr=None)  
# spec = librosa.feature.mfcc(y=wav, sr=fs, n_mfcc=13, n_fft=nfft*4, hop_length=nfft)    
# l_spec = np.shape(spec)[1]  
# h_spec = np.shape(spec)[0]  
# spec_matrix = np.zeros([len(y), h_spec, l_spec])


# for i in range(0, n):
#     audio_file, fs = librosa.load(wav_path.loc[i], sr=None)
#     spec_matrix[i] = librosa.feature.mfcc(
#         y=audio_file, sr=fs, n_mfcc=13, n_fft=nfft*4, hop_length=nfft)


KeyboardInterrupt: 

In [ ]:
file_names = ['data_'+feature_type+'not_sure_single_into_0_5.h5',
              'label_'+feature_type+'not_sure_single_into_0_5.h5']

for file_name in file_names:
    if not os.path.isfile('../proc_data/' + file_name):
        hf = h5py.File('../proc_data/data_' + feature_type +
                       'not_sure_single_into_0_5.h5', 'w')
        hf.create_dataset('../proc_data/data_' + feature_type + '_majority_labels_not_sure_single_into_0_5',
                          data=spec_matrix)
        hf.close()

        hf = h5py.File('../proc_data/label_' + feature_type +
                       'not_sure_single_into_0_5.h5', 'w')
        hf.create_dataset('../proc_data/label_' + feature_type +
                          '_majority_labels_not_sure_single_into_0_5', data=y)
        hf.close()


In [5]:
save_name = 'not_sure_single_into_0_5'
hf = h5py.File('../proc_data/data_' + feature_type + save_name + '.h5', 'r')
spec_matrix_read = np.array(hf.get('../proc_data/data_' + feature_type + '_majority_labels_' + save_name))
hf.close()
hf = h5py.File('../proc_data/label_' + feature_type + save_name + '.h5', 'r')
y = np.array(hf.get('../proc_data/label_' + feature_type + '_majority_labels_' + save_name))
hf.close()

In [6]:
spec_matrix_db = np.zeros_like(spec_matrix_read)
for i, spec in enumerate(spec_matrix_read):
    spec_matrix_db[i] = librosa.power_to_db(spec,ref=np.max)

In [7]:
def get_model():
    X_train, X_test, y_train, y_test = train_test_split(spec_matrix_db, y, test_size=0.33, random_state=45)
    
    X_train_norm = (X_train - np.mean(X_train)) / np.std(X_train)
    X_test_norm = (X_test - np.mean(X_train)) / np.std(X_train)

    # number of files, number of time steps , number of features
    X_train_tf = X_train_norm.reshape(
        X_train_norm.shape[0], X_train_norm.shape[1], X_train_norm.shape[2])
    X_test_tf = X_test_norm.reshape(
        X_test_norm.shape[0], X_test_norm.shape[1], X_test_norm.shape[2])

    
    model = Sequential()
    
    model.add(Conv1D(32, kernel_size=3, activation='relu',
              input_shape=(X_train_tf.shape[1], X_train_tf.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    model.fit(X_train_tf, y_train, batch_size=None, epochs=3,
              verbose=1, class_weight={0: 1., 1: 10.})
    
    loss, acc = model.evaluate(X_test_tf, y_test, batch_size=None, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', acc)
    

    
    return model,np.mean(X_train),np.std(X_train)

In [8]:
def get_model2():
    X_train, X_test, y_train, y_test = train_test_split(
        spec_matrix_db, y, test_size=0.33, random_state=45)

    X_train_norm = (X_train - np.mean(X_train)) / np.std(X_train)
    X_test_norm = (X_test - np.mean(X_train)) / np.std(X_train)

    # reshape to add the channels dimension
    X_train_norm = np.expand_dims(X_train_norm, axis=3)
    X_test_norm = np.expand_dims(X_test_norm, axis=3)

    model = Sequential()

    model.add(Conv2D(filters=8, kernel_size=(8, 10), strides=(2, 2), activation='relu',
              input_shape=(X_train_norm.shape[1], X_train_norm.shape[2], X_train_norm.shape[3])))
    model.add(Flatten())
    model.add(Dense(3, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    model.fit(X_train_norm, y_train, batch_size=None, epochs=3,
              verbose=1, class_weight={0: 1., 1: 10.})

    loss, acc = model.evaluate(X_test_norm, y_test, batch_size=None, verbose=0)
    print('Test loss:', loss)
    print('Test accuracy:', acc)

    return model, np.mean(X_train), np.std(X_train)


In [9]:
model,mean,std = get_model()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


2023-04-21 12:38:34.909618: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-21 12:38:34.909645: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-21 12:38:34.909665: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (parikshit-g55500): /proc/driver/nvidia/version does not exist
2023-04-21 12:38:34.910913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/3


2023-04-21 12:38:35.247965: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 30529356 exceeds 10% of free system memory.


1668/1668 [==============================] - 3s 2ms/step - loss: 1.1367 - accuracy: 0.6893
Epoch 2/3
1668/1668 [==============================] - 3s 2ms/step - loss: 1.0778 - accuracy: 0.7164
Epoch 3/3
1668/1668 [==============================] - 3s 2ms/step - loss: 1.0572 - accuracy: 0.7280
Test loss: 0.5474075675010681
Test accuracy: 0.7593290209770203


INFO:tensorflow:Assets written to: /tmp/tmpuhf7pzp7/assets


INFO:tensorflow:Assets written to: /tmp/tmpuhf7pzp7/assets
2023-04-21 12:38:46.930761: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-04-21 12:38:46.930795: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-04-21 12:38:46.932203: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpuhf7pzp7
2023-04-21 12:38:46.933583: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-21 12:38:46.933603: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpuhf7pzp7
2023-04-21 12:38:46.938321: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-04-21 12:38:46.939833: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-04-21 12:38:46.982003: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

In [10]:
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [11]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('mean.pkl', 'wb') as f:
    pickle.dump(mean, f)

with open('std.pkl', 'wb') as f:
    pickle.dump(std, f)


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv1d
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dropout
.........vars
......flatten
.........vars
......max_pooling1d
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
variables.h5                                   2023-04-21 12:39:20       167472
config.json                                    2023-04-21 12:39:20         2435
metadata.json                                  2023-04-21 12:39:20           64


In [2]:
import pickle

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('mean.pkl', 'rb') as f:
    mean = pickle.load(f)

with open('std.pkl', 'rb') as f:
    std = pickle.load(f)

Keras model archive loading:
File Name                                             Modified             Size
variables.h5                                   2023-04-21 12:39:20       167472
config.json                                    2023-04-21 12:39:20         2435
metadata.json                                  2023-04-21 12:39:20           64
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......conv1d
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dropout
.........vars
......flatten
.........vars
......max_pooling1d
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars


In [3]:
wav_root = '../data/audio_1sec/'

labels = pd.read_csv('../labels/audio_1sec.csv',  header=None, names=["path", "yes", "no", "not_sure","subject_set"])
labels['path'] = labels['path'].astype(str) + '.wav'

labels['res'] = (labels['yes'].astype(int)*1 + labels['not_sure'].astype(int)*0.5) / (labels['yes'] + labels['no'] + labels['not_sure']) >= 0.5
n = len(labels.index.values.tolist())

y = np.zeros((n, 2))

y[:,1] = np.array(np.array(labels['res']).astype(int)).astype(int)
y[:,0] = 1-y[:,1].astype(int)

NameError: name 'pd' is not defined

In [14]:
import IPython.display as ipd
import pyaudio

audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 8000
RECORD_SECONDS = 5
win_len = 0.1


for i in range(0, 10):
    win_len = 0.1
    nfft = int(win_len * 8000)
    
    for j in range(int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    # Convert the recorded audio to numpy array
    audio_np = np.frombuffer(b"".join(frames), dtype=np.int16)

    
    spec = librosa.feature.mfcc(y=wav, sr=fs, n_mfcc=13,
                                n_fft=nfft*4, hop_length=nfft)

    spec_norm = (spec - mean) / std

    spec_reshaped = spec_norm.reshape(
        1, spec_norm.shape[0], spec_norm.shape[1])

    y_pred = model.predict(spec_reshaped)
    print(y_pred)
    y_pred_label = np.argmax(y_pred, axis=1)

    print("Predicted class:", y_pred_label[0], "for file:", p)

    # Display and play the predicted audio file
    ipd.display(ipd.Audio(wav, rate=fs))


1/1 [==============================] - 0s 87ms/step
[[0.07672881 0.9232711 ]]
Predicted class: 1 for file: ../data/audio_1sec/604290093.wav


1/1 [==============================] - 0s 21ms/step
[[0.05839773 0.94160223]]
Predicted class: 1 for file: ../data/audio_1sec/604390974.wav


1/1 [==============================] - 0s 35ms/step
[[0.7154604  0.28453958]]
Predicted class: 0 for file: ../data/audio_1sec/88700038.wav


1/1 [==============================] - 0s 35ms/step
[[0.9794693 0.0205307]]
Predicted class: 0 for file: ../data/audio_1sec/604441500.wav


1/1 [==============================] - 0s 32ms/step
[[0.1212365 0.8787635]]
Predicted class: 1 for file: ../data/audio_1sec/604240573.wav


1/1 [==============================] - 0s 32ms/step
[[0.97616154 0.02383845]]
Predicted class: 0 for file: ../data/audio_1sec/604420459.wav


1/1 [==============================] - 0s 21ms/step
[[0.11297895 0.88702106]]
Predicted class: 1 for file: ../data/audio_1sec/604320940.wav


1/1 [==============================] - 0s 32ms/step
[[0.9789675  0.02103251]]
Predicted class: 0 for file: ../data/audio_1sec/604422189.wav


1/1 [==============================] - 0s 32ms/step
[[0.97575945 0.02424053]]
Predicted class: 0 for file: ../data/audio_1sec/604442176.wav


1/1 [==============================] - 0s 30ms/step
[[0.7151689 0.2848311]]
Predicted class: 0 for file: ../data/audio_1sec/76190004.wav


In [15]:
import serial

# replace 'COM4' with the serial port where your Arduino is connected
arduino = serial.Serial('/dev/ttyACM0', 9600)

while True:
    user_input = input("Enter 1 to turn on LED or 0 to turn off LED: ")
    arduino.write(user_input.encode())


SerialException: [Errno 13] could not open port /dev/ttyACM0: [Errno 13] Permission denied: '/dev/ttyACM0'

In [26]:
from numba import jit

@jit(nopython=True)
def add(x, y):
    return x + y

print(add(1, 2))


AttributeError: module 'numba' has no attribute 'core'